In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [4]:
# Importing important libraries

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

2026-01-22 14:27:03.731195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769092023.960005      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769092024.028156      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769092024.602587      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769092024.602627      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769092024.602630      55 computation_placer.cc:177] computation placer alr

## 1. Importing Data

In [5]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

* data is balanced

In [8]:
### Map the sentiment values to numeric
df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)

/tmp/ipykernel_55/3525237038.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace = True)


In [9]:
df['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

## 2. Train Test Split

In [10]:
from sklearn.model_selection import train_test_split

# X = Features, y = Target/Label
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'],
                                                    test_size=0.2, random_state=42)

In [11]:
print(X_train.shape)
print(X_test.shape)

(40000,)
(10000,)


## 3. Data Preprocessing

### Step 1: Text → Word Indices (Tokenizer)

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 10000, oov_token = "<OOV>")
tokenizer.fit_on_texts(X_train)

X_tran_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

#### Why num_words=10000?

IMDB reviews have huge vocabulary  
Most sentiment info comes from top frequent words  
Limits noise and memory usage

### Example:
"I loved this movie"
→ [14, 213, 9, 27]

### Step 2: Padding (Very Important for LSTM)

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_tran_seq, maxlen=200, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=200, padding='post')

#### Why maxlen=200?

Most IMDB reviews are long  
But sentiment is usually clear early  
200 is a good tradeoff

In [14]:
print(X_test_pad)

[[   1  697    6 ...  996  720  156]
 [  45  201 1568 ...    8 9051 8495]
 [   2   20  921 ...    0    0    0]
 ...
 [  20 2069  204 ...  126  201 3242]
 [  15  226   15 ...    0    0    0]
 [  11  213 8344 ...    0    0    0]]


## 4. Build LSTM Model 

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.layers import Input

# Build the model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=32))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(256, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

I0000 00:00:1769092051.024827      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1769092051.028654      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


### Line by line code explanation
* Creates a Sequential model, meaning layers will be stacked one after another in a linear order.
* Converts each word index into a 32-dimensional dense vector, learning word meanings for the top 10,000 words, with each review padded to 200 words.
* Uses an LSTM layer with 32 memory units to learn word order and context, while dropping 20% of inputs and recurrent connections to reduce overfitting.
* Adds a fully connected layer that learns non-linear combinations of LSTM features using the ReLU activation.
* Outputs a single probability value (0–1) using sigmoid activation to classify the review as positive or negative.

#### 🧠 One-Line Summary of the Whole Model

Words are converted into vectors, context is learned using LSTM, features are refined using a dense layer, and final sentiment is predicted using sigmoid.

In [16]:
## Just to see the parameters and layers in our model
model.build(input_shape=(None, 200))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │         8,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 337,025 (1.29 MB)

 Trainable params: 337,025 (1.29 MB)

 Non-trainable params: 0 (0.00 B)

## 4. Compile the model

In [17]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

#### 🔧 model.compile() — What It Really Does

* compile() tells Keras how to train the model  
– how to measure error,  
– how to update weights,  
– and what to monitor.

#### 1️⃣ Loss Function
loss="binary_crossentropy"
##### What is a loss function?
* A loss function measures how wrong the model’s prediction is compared to the true label.

#### 2️⃣ Optimizer
optimizer="adam"
##### What is an optimizer?
* An optimizer updates the model’s weights to minimize the loss.
* Loss → Gradients → Weight updates

#### 3️⃣ Metrics
metrics=["accuracy"]
##### What is a metric?
* A metric is NOT used for training — it is used for monitoring performance.

Accuracy tells:

“How often is the model correct?”

## 5. Callbacks (training control)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Stop training if validation loss does not improve
early_stopping = EarlyStopping(
    monitor="val_loss",        # Watch validation loss
    patience=3,                # Stop after 3 epochs with no improvement
    restore_best_weights=True  # Revert to best model weights
)

# Save the best model during training
model_checkpoint = ModelCheckpoint(
    "best_lstm_model.h5",      # File to save the model
    monitor="val_loss",        # Save based on validation loss
    save_best_only=True,       # Only save the best model
    verbose=1
)

#### EarlyStopping

* EarlyStopping halts training when validation loss stops improving, preventing overfitting and saving computational resources.

#### ModelCheckpoint

* ModelCheckpoint automatically saves the model with the best validation performance, ensuring the most generalizable version is preserved.

## 6. Train the model

In [19]:
history = model.fit(
    X_train_pad,               # Training input data (padded sequences)
    y_train,                   # Training labels
    epochs=5,                 # Maximum number of epochs
    batch_size=64,             # Number of samples per training step
    validation_split=0.2,      # Validation data
    callbacks=[early_stopping, model_checkpoint],
    verbose=1                  # Show training progress
)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - accuracy: 0.5590 - loss: 0.6685
Epoch 1: val_loss improved from inf to 0.58158, saving model to best_lstm_model.h5


500/500 ━━━━━━━━━━━━━━━━━━━━ 179s 348ms/step - accuracy: 0.5591 - loss: 0.6684 - val_accuracy: 0.6754 - val_loss: 0.5816
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - accuracy: 0.7301 - loss: 0.5317
Epoch 2: val_loss did not improve from 0.58158
500/500 ━━━━━━━━━━━━━━━━━━━━ 173s 346ms/step - accuracy: 0.7300 - loss: 0.5318 - val_accuracy: 0.6401 - val_loss: 0.5979
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - accuracy: 0.7108 - loss: 0.5268
Epoch 3: val_loss improved from 0.58158 to 0.45143, saving model to best_lstm_model.h5


500/500 ━━━━━━━━━━━━━━━━━━━━ 173s 346ms/step - accuracy: 0.7109 - loss: 0.5267 - val_accuracy: 0.8105 - val_loss: 0.4514
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - accuracy: 0.8047 - loss: 0.4423
Epoch 4: val_loss improved from 0.45143 to 0.43880, saving model to best_lstm_model.h5


500/500 ━━━━━━━━━━━━━━━━━━━━ 173s 347ms/step - accuracy: 0.8047 - loss: 0.4423 - val_accuracy: 0.8173 - val_loss: 0.4388
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.8303 - loss: 0.4087
Epoch 5: val_loss improved from 0.43880 to 0.42442, saving model to best_lstm_model.h5


500/500 ━━━━━━━━━━━━━━━━━━━━ 172s 345ms/step - accuracy: 0.8302 - loss: 0.4087 - val_accuracy: 0.8317 - val_loss: 0.4244


#### 📌 What happens during training
* Data flows: Embedding → LSTM → Dense → Output
* Loss is calculated using binary_crossentropy
* Backpropagation updates Embedding, LSTM, and Dense weights
* Training stops early if validation loss stops improving

## 7. Evaluate the model

In [21]:
loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)

print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Loss: 0.4139
Validation Accuracy: 0.8406


## 8. Make predictions (optional)

In [22]:
# Make predictions (optional)
predictions = model.predict(X_test_pad)

# Convert probabilities to class labels
prediction_labels = (predictions > 0.5).astype(int)

313/313 ━━━━━━━━━━━━━━━━━━━━ 32s 100ms/step


In [23]:
predictions

array([[0.6749363 ],
       [0.76927453],
       [0.2040222 ],
       ...,
       [0.9598855 ],
       [0.20397727],
       [0.8650386 ]], dtype=float32)

## 9. Build Predictive System

In [26]:
def predict_sentiment(review):
    # Convert text to sequence of word indices
    sequence = tokenizer.texts_to_sequences([review])

    # Pad sequence to match training input length
    padded_sequence = pad_sequences(sequence, maxlen=200, padding='post')

    # Predict sentiment probability
    prediction = model.predict(padded_sequence)

    # Convert probability to label
    sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"

    return sentiment

## 10. Example Usage

In [27]:
## predict sentiment for a review
new_review = "This movies was fantastic"
customer_sentiment = predict_sentiment(new_review)
print(customer_sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
Positive


In [32]:
## predict sentiment for a review
new_review = "This movies was just awesome"
customer_sentiment = predict_sentiment(new_review)
print(customer_sentiment)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Positive


## 11. Save the Model & tokenizer

In [33]:
# Save model
model.save("sentiment_model.keras")

In [34]:
# Save tokenizer
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)